In [ ]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter

from mpl_toolkits.basemap import Basemap
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'